In [22]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import re
import string
import nltk
from nltk import pos_tag
from nltk.corpus import stopwords, wordnet 
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from nltk.tokenize import word_tokenize, sent_tokenize

from sklearn.preprocessing import LabelEncoder 
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV, GridSearchCV

from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import plot_roc_curve

from wordcloud import WordCloud

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_json(r"Cell_Phones_and_Accessories.json")

In [3]:
df.head()

Rating  Rev_verify  Review_Date          IC  \
0          5        True   09 1, 2015  B009XD5TPQ   
1          5        True   01 9, 2016  B016MF3P3K   
10         1        True  01 14, 2016  B00IJJCQBA   
100        5       False  06 21, 2017  B00NIJOGOG   
1000       3        True  06 10, 2015  B00MQYS97Y   

                                              Prod_meta   Reviewer_Name  \
0                                                  None  Sunny  Zoeller   
1                                                  None   Denise Lesley   
10                         {'Color:': ' Black / Black'}       Stephanie   
100   {'Color:': ' Rose Gold [6+] CHOOSE CORRECT SIZ...              SG   
1000                                               None           Linda   

                                                 Review  \
0     Bought it for my husband. He's very happy with it   
1     Great screen protector.  Doesn't even seem as ...   
10    To tight on my phone and the bottom piece was ...   
100   Very good and superior quality, looks great. M...   
1000  The charger is not working, however ! The comp...   

                           Rev_summ  Review_timestamp Useful Prod_img  
0              He's very happy with        1441065600   None     None  
1                        Five Stars        1452297600   None     None  
10                         One Star        1452729600   None     None  
100   Very nice and good quality!!!        1498003200   None     None  
1000                      Not happy        1433894400   None     None

In [4]:
df.shape

(760450, 11)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 760450 entries, 0 to 99999
Data columns (total 11 columns):
Rating              760450 non-null int64
Rev_verify          760450 non-null bool
Review_Date         760450 non-null object
IC                  760450 non-null object
Prod_meta           407826 non-null object
Reviewer_Name       760359 non-null object
Review              759920 non-null object
Rev_summ            760095 non-null object
Review_timestamp    760450 non-null int64
Useful              62200 non-null object
Prod_img            18194 non-null object
dtypes: bool(1), int64(2), object(8)
memory usage: 64.5+ MB


In [6]:
df.isna().sum()

Rating                   0
Rev_verify               0
Review_Date              0
IC                       0
Prod_meta           352624
Reviewer_Name           91
Review                 530
Rev_summ               355
Review_timestamp         0
Useful              698250
Prod_img            742256
dtype: int64

In [7]:
df.drop(['Review_Date','Prod_meta','Reviewer_Name','Review_timestamp','Useful','Prod_img'],axis=1,inplace=True)

In [8]:
df.dropna(inplace=True)

In [9]:
df.isna().sum()

Rating        0
Rev_verify    0
IC            0
Review        0
Rev_summ      0
dtype: int64

In [10]:
df.duplicated().sum()

5254

In [11]:
df = df.drop_duplicates(keep=False)

In [12]:
df.duplicated().sum()

0

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 750097 entries, 0 to 99999
Data columns (total 5 columns):
Rating        750097 non-null int64
Rev_verify    750097 non-null bool
IC            750097 non-null object
Review        750097 non-null object
Rev_summ      750097 non-null object
dtypes: bool(1), int64(1), object(3)
memory usage: 29.3+ MB


In [14]:
df.head()

Rating  Rev_verify          IC  \
0          5        True  B009XD5TPQ   
1          5        True  B016MF3P3K   
10         1        True  B00IJJCQBA   
100        5       False  B00NIJOGOG   
1000       3        True  B00MQYS97Y   

                                                 Review  \
0     Bought it for my husband. He's very happy with it   
1     Great screen protector.  Doesn't even seem as ...   
10    To tight on my phone and the bottom piece was ...   
100   Very good and superior quality, looks great. M...   
1000  The charger is not working, however ! The comp...   

                           Rev_summ  
0              He's very happy with  
1                        Five Stars  
10                         One Star  
100   Very nice and good quality!!!  
1000                      Not happy

In [15]:
# We can combine Review and Rev_summ
df['review_summary'] = df[['Review','Rev_summ']].apply(lambda x:' '.join(x), axis=1)

In [16]:
# Removing Review and Rev_summ
df.drop(['Review','Rev_summ'],axis=1,inplace=True)
df.head()

Rating  Rev_verify          IC  \
0          5        True  B009XD5TPQ   
1          5        True  B016MF3P3K   
10         1        True  B00IJJCQBA   
100        5       False  B00NIJOGOG   
1000       3        True  B00MQYS97Y   

                                         review_summary  
0     Bought it for my husband. He's very happy with...  
1     Great screen protector.  Doesn't even seem as ...  
10    To tight on my phone and the bottom piece was ...  
100   Very good and superior quality, looks great. M...  
1000  The charger is not working, however ! The comp...

In [17]:
df.IC.value_counts()

B00VH88CJ0    1079
B005NF5NTK    1066
B00P7N0320     991
B018JW3EOY     990
B00X5RV14Y     984
B019PV2I3G     983
B00MQSMDYU     877
B00G7UY3EG     861
B00M51DDT2     828
B00QN1T6NM     752
B00D856NOG     741
B00IGISUTG     728
B0092KJ9BU     714
B00BT8L2MW     702
B00AANQLRI     690
B00UCZGS6S     670
B00NH12EHQ     665
B00MXWFUQC     655
B00M6QODH2     637
B014EB532U     637
B00UH3L82Y     631
B009YCP1LS     599
B0085JRQZU     590
B00JH88NHI     584
B004G1L52Q     571
B00M5JKZY0     568
B00P933OJC     562
B002BBJMO6     557
B0194WDVHI     555
B00NP9IZIM     553
              ... 
B000WYYUHI       1
B000ZQ8HZO       1
B00656BMF0       1
B000X1OGGU       1
B00UAJJJEY       1
B00BMEQ1I8       1
B016BCHP62       1
B00IRS2FOG       1
B01DPAL8ZM       1
B00133TOJG       1
B00MHELLLE       1
B00OLF6F86       1
B0014YXWV0       1
B01DHH3K7W       1
B00JVEPKJC       1
B009FKP1ZU       1
B0013N40UO       1
B016I8KS0O       1
B00CO5DDYE       1
B00009RUHV       1
B00IN5KZXG       1
B01CAFT19I  

In [18]:
df.Rating.value_counts()

5    468432
4    122846
3     65945
1     54589
2     38285
Name: Rating, dtype: int64

In [19]:
# Find out avg, max and min ratings
df['min_rating']= df.groupby(['IC'])['Rating'].transform(lambda x: x.min())
df['max_rating']= df.groupby(['IC'])['Rating'].transform(lambda x: x.max())
df['avg_rating']= df.groupby(['IC'])['Rating'].transform(lambda x: x.mean())

df.head()

Rating  Rev_verify          IC  \
0          5        True  B009XD5TPQ   
1          5        True  B016MF3P3K   
10         1        True  B00IJJCQBA   
100        5       False  B00NIJOGOG   
1000       3        True  B00MQYS97Y   

                                         review_summary  min_rating  \
0     Bought it for my husband. He's very happy with...           2   
1     Great screen protector.  Doesn't even seem as ...           1   
10    To tight on my phone and the bottom piece was ...           1   
100   Very good and superior quality, looks great. M...           1   
1000  The charger is not working, however ! The comp...           2   

      max_rating  avg_rating  
0              5    4.500000  
1              5    3.636364  
10             5    4.150376  
100            5    4.297872  
1000           5    4.518519

In [20]:
df.reset_index(drop=True, inplace=True)
df.head()

Rating  Rev_verify          IC  \
0       5        True  B009XD5TPQ   
1       5        True  B016MF3P3K   
2       1        True  B00IJJCQBA   
3       5       False  B00NIJOGOG   
4       3        True  B00MQYS97Y   

                                      review_summary  min_rating  max_rating  \
0  Bought it for my husband. He's very happy with...           2           5   
1  Great screen protector.  Doesn't even seem as ...           1           5   
2  To tight on my phone and the bottom piece was ...           1           5   
3  Very good and superior quality, looks great. M...           1           5   
4  The charger is not working, however ! The comp...           2           5   

   avg_rating  
0    4.500000  
1    3.636364  
2    4.150376  
3    4.297872  
4    4.518519

In [21]:
from nltk.corpus import wordnet

def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    

In [23]:
def clean_text(text):
    text = text.lower()
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    text = [word for word in text if not any(b.isdigit() for b in word)]
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]
    text = [t for t in text if len(t) > 0]
    pos_tags = pos_tag(text)
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    text = [t for t in text if len(t) > 1]
    text = " ".join(text)
    return(text)

In [24]:
df["review_cleaned"] = df["review_summary"].apply(lambda x: clean_text(x))

In [26]:
df.head()

Rating  Rev_verify          IC  \
0       5        True  B009XD5TPQ   
1       5        True  B016MF3P3K   
2       1        True  B00IJJCQBA   
3       5       False  B00NIJOGOG   
4       3        True  B00MQYS97Y   

                                      review_summary  min_rating  max_rating  \
0  Bought it for my husband. He's very happy with...           2           5   
1  Great screen protector.  Doesn't even seem as ...           1           5   
2  To tight on my phone and the bottom piece was ...           1           5   
3  Very good and superior quality, looks great. M...           1           5   
4  The charger is not working, however ! The comp...           2           5   

   avg_rating                                     review_cleaned  
0    4.500000               bought husband he's happy he's happy  
1    3.636364  great screen protector even seem though five star  
2    4.150376             tight phone bottom piece miss one star  
3    4.297872  good superior quality look great wife use last...  
4    4.518519  charger work however company send replacement ...

In [33]:
# function to plot most frequent terms
def freq_words(x, terms = 30):
    all_words = x.split()

    fdist = nltk.FreqDist(all_words)
    words_df = pd.DataFrame({'word':list(fdist.keys()), 'count':list(fdist.values())})

    # selecting top 20 most frequent words
    d = words_df.nlargest(columns="count", n = terms) 
    frequent_words=[]
    for word in d['word']:
        frequent_words.append(word)
    return frequent_words
    

In [35]:
freq_words(df["review_cleaned"][1])

['great', 'screen', 'protector', 'even', 'seem', 'though', 'five', 'star']

In [36]:
df['summary_keywords']=df["review_cleaned"].apply(freq_words)

In [37]:
df.head()

Rating  Rev_verify          IC  \
0       5        True  B009XD5TPQ   
1       5        True  B016MF3P3K   
2       1        True  B00IJJCQBA   
3       5       False  B00NIJOGOG   
4       3        True  B00MQYS97Y   

                                      review_summary  min_rating  max_rating  \
0  Bought it for my husband. He's very happy with...           2           5   
1  Great screen protector.  Doesn't even seem as ...           1           5   
2  To tight on my phone and the bottom piece was ...           1           5   
3  Very good and superior quality, looks great. M...           1           5   
4  The charger is not working, however ! The comp...           2           5   

   avg_rating                                     review_cleaned  \
0    4.500000               bought husband he's happy he's happy   
1    3.636364  great screen protector even seem though five star   
2    4.150376             tight phone bottom piece miss one star   
3    4.297872  good superior quality look great wife use last...   
4    4.518519  charger work however company send replacement ...   

                                    summary_keywords  
0                     [he's, happy, bought, husband]  
1  [great, screen, protector, even, seem, though,...  
2     [tight, phone, bottom, piece, miss, one, star]  
3  [look, good, quality, great, superior, wife, u...  
4  [wire, happy, charger, work, however, company,...

In [38]:
# Final dataframe
df_final=df[['IC','min_rating','max_rating','avg_rating','summary_keywords']]

In [39]:
df_final.head()

IC  min_rating  max_rating  avg_rating  \
0  B009XD5TPQ           2           5    4.500000   
1  B016MF3P3K           1           5    3.636364   
2  B00IJJCQBA           1           5    4.150376   
3  B00NIJOGOG           1           5    4.297872   
4  B00MQYS97Y           2           5    4.518519   

                                    summary_keywords  
0                     [he's, happy, bought, husband]  
1  [great, screen, protector, even, seem, though,...  
2     [tight, phone, bottom, piece, miss, one, star]  
3  [look, good, quality, great, superior, wife, u...  
4  [wire, happy, charger, work, however, company,...

In [40]:
df_final.to_json('product_review.json')